In [1]:
from pygmsh.built_in.geometry import Geometry
from pygmsh import generate_mesh
import meshio
import dolfin
import dolfin.io
from dolfin.plotting import plot
import matplotlib

# Define input data
from ufl import SpatialCoordinate, inner, grad, lhs, rhs, dot, exp, Measure, dx, ds
from dolfin.fem import assemble_scalar
from dolfin import FunctionSpace, TrialFunction, TestFunction, DirichletBC, Function, solve, cpp,fem    

In [4]:
# Read the file generated by gmsh
mesh = meshio.read("Models/poission_subdomain.msh")
points, cells, cell_data, field_data = mesh.points, mesh.cells, mesh.cell_data, mesh.field_data

In [5]:
meshio.write("poisson_subdomain_mesh.xdmf", 
             meshio.Mesh(
                points=points[:,:2],# Converting to 2D
                cells={"triangle": cells["triangle"]},
                cell_data={"triangle": {"subdomain": cell_data["triangle"]["gmsh:physical"]}},
                field_data=field_data
             )
            )

meshio.write("poisson_subdomain_boundary.xdmf", meshio.Mesh(
    points=points[:,:2],# Converting to 2D
    cells={"line": cells["line"]},
    cell_data={"line": {"boundaries": cell_data["line"]["gmsh:physical"]}}
))

In [7]:
with dolfin.io.XDMFFile(dolfin.MPI.comm_world, "poisson_subdomain_mesh.xdmf") as xdmf_infile:
    mesh = xdmf_infile.read_mesh( dolfin.MPI.comm_world, dolfin.cpp.mesh.GhostMode.none)
    mvc_subdomain = xdmf_infile.read_mvc_size_t(mesh, "subdomain")
    tag_info = xdmf_infile.read_information_int()

with dolfin.io.XDMFFile(dolfin.MPI.comm_world, "poisson_subdomain_boundary.xdmf") as xdmf_infile:
    mvc_boundaries = xdmf_infile.read_mvc_size_t(mesh, "boundaries")

print("Constructing MeshFunction from MeshValueCollection")
domains = dolfin.cpp.mesh.MeshFunctionSizet(mesh, mvc_subdomain, 0)
boundaries = dolfin.cpp.mesh.MeshFunctionSizet(mesh, mvc_boundaries, 0)

AttributeError: 'XDMFFile' object has no attribute 'read_information_int'

In [ ]:
a0 = 1.0
a1 = 0.01
x = SpatialCoordinate(mesh)
g_L = exp(- 10*(- pow(x[1] - 0.5, 2)))
g_R = 1.0
f = 1.0

# Define function space and basis functions
V = FunctionSpace(mesh, ("CG", 2))
u = TrialFunction(V)
v = TestFunction(V)

u5 = Function(V)
with u5.vector().localForm() as bc_local:
    bc_local.set(5.0)

u0 = Function(V)
with u0.vector().localForm() as bc_local:
    bc_local.set(0.0)


# Define Dirichlet boundary conditions at top and bottom boundaries
bcs = [DirichletBC(V, u5, np.where(boundaries.values == tag_info['TOP'])[0]),
       DirichletBC(V, u0, np.where(boundaries.values == tag_info['BOTTOM'])[0])]

dx = dx(subdomain_data=domains)
ds = ds(subdomain_data=boundaries)

# Define variational form
F = (inner(a0*grad(u), grad(v))*dx(tag_info['DOMAIN']) + 
    inner(a1*grad(u), grad(v))*dx(tag_info['OBSTACLE'])
     - g_L*v*ds(tag_info['LEFT']) 
     - g_R*v*ds(tag_info['RIGHT'])
     - f*v*dx(tag_info['DOMAIN']) 
     - f*v*dx(tag_info['OBSTACLE']))


# Separate left and right hand sides of equation
a, L = lhs(F), rhs(F)

# Solve problem
u = Function(V)
solve(a == L, u, bcs)

with dolfin.io.XDMFFile(dolfin.MPI.comm_world, "output.xdmf") as xdmf_outfile:
    xdmf_outfile.write(u)

pass